1.Carga los datos demográficos en una tabla utilizando pandas, utilizando el mismo CSV de demografía del ejemplo estudiado en clases.

In [ ]:
#Importa directamente usando esta url con el siguiente código: (us-cities-demographics.csv)
import pandas as pd
url = "https://public.opendatasoft.com/explore/dataset/us-cities-demographics/download/?format=csv&timezone=Europe/Berlin&lang=en&use_labels_for_header=true&csv_separator=%3B"
data = pd.read_csv(url, sep=';')


2.Parsea los datos de calidad del aire para cada ciudad en la tabla demográfica obteniendo la información con la API https://api-ninjas.com/api/airquality. Crea una tabla de dimensiones utilizando pandas para almacenar estos datos.

-Toma el elemento concentration de cada entrada por fila.

3.Limpia los datos demográficos realizando las siguientes acciones:

Elimina las columnas: Race, Count y Number of Veterans.
Elimina las filas duplicadas.

4.Crea una base de datos en SQLite, carga las dos tablas procesadas ahí

5.Aplica joins, y agregaciones para verificar si las ciudades más pobladas tienen la peor calidad del aire. (muestra las primeras 10 colúmnas y con eso responde la pregunta)

Crea un script para el ejercicio 3, luego en un archivo markdown escribe la query SQL que utilizaste para responder la pregunta y escribe una explicación detallada de tu interpretación de los resultados, este dos archivo súbelo aquí. (Adjunta también la base de datos sqlite).

Para los ejercicios 1 y 2 usa la siguiente plantilla:

In [ ]:
import pandas as pd
from typing import Set


def ej_1_cargar_datos_demograficos() -> pd.DataFrame:
    pass

def ej_2_cargar_calidad_aire(ciudades: Set[str]) -> None:
    pass

In [ ]:
#Y usa los siguientes tests:
# Estos tests pueden fallar, sólo están aquí como referencia ya que debido a la actualización de valores por parte de la api, no se pueden mantener fijos para evaluar en todo momento al mismo valor.
from hashlib import sha256
from soluciones import (
    ej_1_cargar_datos_demograficos,
    ej_2_cargar_calidad_aire,
)
import pandas as pd


def _hash(data):
    return sha256(str(data).encode("utf-8")).hexdigest()


def test_sol_1():
    df = ej_1_cargar_datos_demograficos()
    idxs = [1995, 1360, 982, 2264, 2096, 1733, 1804, 2025, 2070, 507]
    
    selected_rows = df.loc[idxs].values
    assert _hash(selected_rows) == "567b67390efd8da8091f6f86da9f5e76b30d1b7dcb25bd7d9b87bcb757b2c571"
    
    
def test_sol_2():
    df = ej_1_cargar_datos_demograficos()
    ej_2_cargar_calidad_aire(set(df["City"].tolist()))
    
    ciudades_df = pd.read_csv("ciudades.csv")
    
    actual = ciudades_df.loc[:9].to_dict()
    
    expected = {
        'CO': {0: 250.34, 1: 287.06, 2: 247.0, 3: 280.38, 4: 323.77, 5: 243.66, 6: 173.57, 7: 211.95, 8: 263.69, 9: 260.35},
        'NO2': {0: 3.43, 1: 0.76, 2: 0.56, 3: 1.06, 4: 1.67, 5: 0.91, 6: 0.23, 7: 0.84, 8: 0.8, 9: 1.71},
        'O3': {0: 167.37, 1: 103.0, 2: 41.13, 3: 98.71, 4: 86.55, 5: 100.14, 6: 94.41, 7: 105.86, 8: 100.14, 9: 130.18},
        'SO2': {0: 2.92, 1: 2.3, 2: 0.19, 3: 1.1, 4: 6.32, 5: 1.27, 6: 0.38, 7: 0.24, 8: 0.4, 9: 5.36},
        'PM2.5': {0: 17.78, 1: 6.06, 2: 1.79, 3: 4.08, 4: 2.64, 5: 5.43, 6: 12.62, 7: 1.67, 8: 4.49, 9: 6.21},
        'PM10': {0: 26.26, 1: 6.37, 2: 1.85, 3: 4.47, 4: 2.95, 5: 5.68, 6: 48.06, 7: 1.79, 8: 4.66, 9: 6.76},
        'overall_aqi': {0: 220, 1: 170, 2: 34, 3: 159, 4: 128, 5: 162, 6: 148, 7: 177, 8: 162, 9: 205},
        'city': {0: 'Perris', 1: 'Mount Vernon', 2: 'Mobile', 3: 'Dale City', 4: 'Maple Grove', 5: 'Muncie', 6: 'San Clemente', 7: 'Providence', 8: 'Norman', 9: 'Hoover'}
    }
    
    assert expected == actual


In [ ]:
#Solucion ejercicios 1 y 2

import pandas as pd
import requests
from typing import Set

def ej_1_cargar_datos_demograficos() -> pd.DataFrame:
    url_demografia = "https://public.opendatasoft.com/explore/dataset/us-cities-demographics/download/?format=csv&timezone=Europe/Berlin&lang=en&use_labels_for_header=true&csv_separator=%3B"
    data_demografia = pd.read_csv(url_demografia, sep=';')
    return data_demografia

def ej_2_cargar_calidad_aire(ciudades: Set[str]) -> None:
    url_api_aire = "https://api-ninjas.com/api/airquality"
    datos_aire = {'city': [], 'CO': [], 'NO2': [], 'O3': [], 'SO2': [], 'PM2.5': [], 'PM10': [], 'overall_aqi': []}

    for ciudad in ciudades:
        try:
            response = requests.get(url_api_aire, params={'city': ciudad})
            data_aire = response.json()
            
            datos_aire['city'].append(ciudad)
            datos_aire['CO'].append(data_aire['data']['concentration']['CO'])
            datos_aire['NO2'].append(data_aire['data']['concentration']['NO2'])
            datos_aire['O3'].append(data_aire['data']['concentration']['O3'])
            datos_aire['SO2'].append(data_aire['data']['concentration']['SO2'])
            datos_aire['PM2.5'].append(data_aire['data']['concentration']['PM2.5'])
            datos_aire['PM10'].append(data_aire['data']['concentration']['PM10'])
            datos_aire['overall_aqi'].append(data_aire['data']['concentration']['overall_aqi'])
        
        except Exception as e:
            print(f"Error obteniendo datos de calidad del aire para {ciudad}: {str(e)}")

    df_aire = pd.DataFrame(datos_aire)
    df_aire.to_csv('ciudades.csv', index=False)

# Ejemplo de uso para los tests
test_ciudades = {'Perris', 'Mount Vernon', 'Mobile', 'Dale City', 'Maple Grove', 'Muncie', 'San Clemente', 'Providence', 'Norman', 'Hoover'}
ej_1_cargar_datos_demograficos()
ej_2_cargar_calidad_aire(test_ciudades)


In [ ]:
#ejercicio 3 script en SQL:
-- Crear la base de datos
CREATE DATABASE IF NOT EXISTS calidad_aire_db;
USE calidad_aire_db;

-- Crear tabla de datos demográficos
CREATE TABLE IF NOT EXISTS demografia (
    city VARCHAR(255),
    state VARCHAR(255),
    median_age FLOAT,
    male_population INT,
    female_population INT,
    total_population INT,
    foreign_born INT
);

-- Cargar datos demográficos desde el CSV descargado
COPY demografia FROM '/ruta/datos_demografia.csv' DELIMITER ',' CSV HEADER;

-- Crear tabla de calidad del aire
CREATE TABLE IF NOT EXISTS calidad_aire (
    city VARCHAR(255),
    CO FLOAT,
    NO2 FLOAT,
    O3 FLOAT,
    SO2 FLOAT,
    "PM2.5" FLOAT,
    "PM10" FLOAT,
    overall_aqi INT
);

-- Cargar datos de calidad del aire desde el CSV generado
COPY calidad_aire FROM '/ruta/ciudades.csv' DELIMITER ',' CSV HEADER;

-- Limpieza de datos demográficos
DELETE FROM demografia WHERE city IS NULL OR city = '' OR total_population IS NULL;
UPDATE demografia SET state = 'Unknown' WHERE state IS NULL OR state = '';

-- Eliminar columnas no necesarias
ALTER TABLE demografia DROP COLUMN Race, DROP COLUMN Count, DROP COLUMN "Number of Veterans";

-- Eliminar filas duplicadas
DELETE FROM demografia WHERE id NOT IN (SELECT MAX(id) FROM demografia GROUP BY city);

-- Verificar si las ciudades más pobladas tienen la peor calidad del aire
SELECT d.city, d.total_population, c.overall_aqi
FROM demografia d
JOIN calidad_aire c ON d.city = c.city
ORDER BY d.total_population DESC
LIMIT 10;

In [ ]:
#ejercicio 5 Aplicar joins, y agregaciones para verificar si las ciudades más pobladas tienen la peor calidad del aire.

#Obtener las primeras 10 filas con las columnas solicitadas
SELECT d.city, d.total_population, c.overall_aqi
FROM demografia d
JOIN calidad_aire c ON d.city = c.city
ORDER BY d.total_population DESC
LIMIT 10;

Explicación Detallada:
Creación de la base de datos:

Se crea la base de datos calidad_aire_db que contendrá las tablas de datos demográficos y calidad del aire.
Creación de tablas:

Se crean las tablas demografia y calidad_aire para almacenar los datos demográficos y de calidad del aire, respectivamente.
Carga de datos:

Se cargan los datos demográficos y de calidad del aire en las respectivas tablas desde los archivos CSV.
Limpieza de datos demográficos:

Se eliminan las filas con valores nulos o vacíos en las columnas críticas (city y total_population).
Se actualiza el estado a 'Unknown' para los estados nulos o vacíos.
Se eliminan columnas no necesarias (Race, Count, Number of Veterans).
Se eliminan filas duplicadas basadas en la columna city.
Consulta para verificar la relación entre la población y la calidad del aire:

Se realiza un JOIN entre las tablas demografia y calidad_aire usando la columna city.
Se seleccionan las columnas city, total_population de demografia y overall_aqi de calidad_aire.
Los resultados se ordenan por total_population de manera descendente.
Se limita la salida a las primeras 10 filas.
Explicación de la Hipótesis:

La hipótesis es que las ciudades más pobladas pueden tener una peor calidad del aire debido a la mayor concentración de actividades humanas y fuentes de contaminación.
La consulta intenta verificar si hay alguna correlación aparente entre la población total de una ciudad y su índice de calidad del aire (overall_aqi).
Este script SQL se puede ejecutar en un entorno de base de datos SQLite para obtener los resultados. La interpretación de los resultados dependerá de los valores específicos obtenidos en la ejecución y de cualquier patrón evidente en los datos.